In [ ]:
# !pip install imutils
# !pip install facenet_pytorch

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.utils import to_categorical
from tensorflow.data import AUTOTUNE
from imutils import paths
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import argparse
import os
from facenet_pytorch import MTCNN
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()

In [ ]:
# help(MTCNN)

In [ ]:
mt = MTCNN(
    image_size = 128,
    select_largest = True,
    selection_method = "probability",
    device = device
)

In [ ]:
BASE_PATH = "../input/deepfake-and-real-images/Dataset"
TRAIN_PATH = os.path.join(BASE_PATH, "Train")
VAL_PATH = os.path.join(BASE_PATH, "Validation")
TEST_PATH = os.path.join(BASE_PATH, "Test")


train_paths = list(paths.list_images(TRAIN_PATH))
print("Training images count = ",len(train_paths))
test_paths = list(paths.list_images(TEST_PATH))
print("Test images count = ",len(test_paths))
val_paths = list(paths.list_images(VAL_PATH))
print("Validating images count = ",len(val_paths))

In [ ]:
def load_images(imagePath):
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels = 3)
#     image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    # image = tf.image.resize(image, config.IMAGE_SIZE)
    stat = tf.strings.regex_full_match(imagePath, r".*Real.*")
    label = 1
    if stat:
        label = 0 
    label = tf.convert_to_tensor(label)
    return (image, label)

In [ ]:
def crop(image, label):
#     imgs = mt(image.numpy())
#     if imgs is None:
#         imgs = image[64:192, 64:192, :]
#     else:
#         imgs = imgs.permute(1, 2, 0).cpu().detach().numpy()
#     imgs = tf.convert_to_tensor(imgs)
    image = tf.image.convert_image_dtype(imgs, dtype=tf.float32)
    return image, label

In [ ]:
BS = 96
trainDS = tf.data.Dataset.from_tensor_slices(train_paths)
trainDS = (trainDS
    .shuffle(len(train_paths))
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .map(lambda x, y: tf.py_function(func=crop,
          inp=[x, y], Tout=(tf.float32, tf.int32)),
          num_parallel_calls=tf.data.AUTOTUNE,
          deterministic=False)
    .batch(BS)
    .prefetch(AUTOTUNE)
    .cache()
)

In [ ]:
BS = 32
valDS = tf.data.Dataset.from_tensor_slices(val_paths)
valDS = (valDS
    .shuffle(len(val_paths))
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .map(lambda x, y: tf.py_function(func=crop,
          inp=[x, y], Tout=(tf.float32, tf.int32)),
          num_parallel_calls=tf.data.AUTOTUNE,
          deterministic=False)
    .batch(BS)
    .prefetch(AUTOTUNE)
    .cache()
)

In [ ]:
BS = 1
testDS = tf.data.Dataset.from_tensor_slices(test_paths)
testDS = (testDS
    .shuffle(len(test_paths))
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .map(lambda x, y: tf.py_function(func=crop,
          inp=[x, y], Tout=(tf.float32, tf.int32)),
          num_parallel_calls=tf.data.AUTOTUNE,
          deterministic=False)
    .batch(BS)
    .prefetch(AUTOTUNE)
    .cache()
)

In [ ]:
for a, b in testDS.take(2):
#     print(a.numpy())
    print(b)
    img = a.numpy()[0]
    plt.imshow(img)
#     display(Image.fromarray(a.numpy()[0], 'RGB'))
#     print("filename: ", a.numpy().decode('utf-8'))
#     print("label:", label.numpy())

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
model = Sequential()

# Pamameters Initialization
input_shape = (128,128,3)
activation = 'relu'
padding = 'same'
droprate = 0.1
epsilon=0.001

model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(filters=16, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))


model.add(Conv2D(filters=32, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=64, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=128, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=256, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

# model.add(Conv2D(filters=512, kernel_size=3, activation=activation, padding=padding))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization(epsilon=epsilon))
# model.add(Dropout(droprate))

# model.add(Conv2D(filters=512, kernel_size=3, activation=activation, padding=padding))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization(epsilon=epsilon))
# model.add(Dropout(droprate))

# model.add(GlobalAveragePooling2D())
model.add(Flatten())
# model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu'))
# model.add(Dropout(0.5))
          
# model.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dropout(droprate))

model.add(Dense(48, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dropout(droprate))
model.add(Dense(1, activation="sigmoid"))
model.summary() # Summary of the architecture

In [ ]:
from tensorflow.keras.optimizers import Adam

#opt = rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss="binary_crossentropy",optimizer=Adam(0.01), metrics=['binary_accuracy'])

In [ ]:
history = model.fit(
    trainDS,
    epochs=10,
#     steps_per_epoch = train_steps,
    validation_data = valDS,
#     validation_steps = valid_steps,
#     callbacks=[PredictionCallback()]
)

In [ ]:
model.save("model.h5")